In [1]:
# from tensorflow.keras.layers import (
#     multiply,
#     concatenate,
# )
# from tensorflow.keras.layers import MaxPooling2D
# from tensorflow.keras.layers import GaussianNoise, GaussianDropout
# from tensorflow.keras.layers import Lambda

# from tensorflow.keras.utils import to_categorical

# import tensorflow.keras.backend as K

In [2]:
# import pickle
# import cv2

# import PIL

# from IPython import display

In [3]:
import tensorflow as tf

# https://github.com/eriklindernoren/Keras-GAN/blob/master/dcgan/dcgan.py
# from keras.datasets import mnist
from tensorflow.keras.layers import (
    Input,
    Dense,
    Reshape,
    Flatten,
    Dropout,
    BatchNormalization,
    Activation,
    ZeroPadding2D,
    LeakyReLU,
    UpSampling2D,
    Conv2D,
)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
import h5py
import pathlib

import time

import math

In [4]:
print("Tensorflow version: ", tf.VERSION)  # 1.15.2
print("Keras version: ", tf.keras.__version__)  # 2.2.4-tf

tf.enable_eager_execution()
print("Is eager execution enabled: ", tf.executing_eagerly())
print("Is there a GPU available: ", tf.test.is_gpu_available())

Tensorflow version:  1.15.2
Keras version:  2.2.4-tf
Is eager execution enabled:  True
Is there a GPU available:  True


In [5]:
path_full_tfrecord = "/data/fp85.tfrecord"
dir_save = "dcgan_85"

In [6]:
# https://medium.com/@moritzkrger/speeding-up-keras-with-tfrecord-datasets-5464f9836c36

BUFFER_SIZE = 1024


def _parse_function(example_proto):
    # Create a description of the features.
    feature_description = {
        "floorplan": tf.io.FixedLenFeature(
            [28, 28, 6], tf.float32, default_value=tf.zeros([28, 28, 6], tf.float32)
        ),
        "plan_id": tf.io.FixedLenFeature([], tf.string, default_value=""),
        "norm_year": tf.io.FixedLenFeature([], tf.float32, default_value=-1.0),
        "sido": tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        "norm_area": tf.io.FixedLenFeature([], tf.float32, default_value=0.0),
        "num_rooms": tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        "num_baths": tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        "brands": tf.io.FixedLenFeature(
            [12], tf.int64, default_value=tf.zeros([12], tf.int64)
        ),
    }

    # Parse the input tf.Example proto using the dictionary above.
    parsed_example = tf.io.parse_single_example(example_proto, feature_description)

    return (parsed_example["floorplan"],)
    # return (parsed_example["floorplan"], parsed_example["brands"])


def _onehot_fp(fp, *rest):
    """
    Replace unit layer with outdoor layer and restrict function space layers indoor
    """
    fp_func = fp[:, :, 1:6] * tf.reshape(fp[:, :, 0], (28, 28, 1))
    fp_out = tf.reshape(1 - fp[:, :, 0], (28, 28, 1))
    return (tf.concat([fp_out, fp_func], axis=2), *rest)


def _rescale_fp(fp, *rest):
    """
    Rescale from [0, 1] to [-1, 1]
    """
    return (fp * 2 - 1, *rest)


def _rescale_fp_brand(fp, brand, *rest):
    """
    Rescale from [0, 1] to [-1, 1]
    """
    return (fp * 2 - 1, brand * 2 - 1, *rest)

def _visualize_fp_onehot(fps):
    # adjusted for different luminance
    channel_to_rgba = np.array(
        [
            [0.0, 0.0, 0.0, 0.0],  # ignore outdoor mask
            [0.0, 0.33, 0.0, 0.0],  # entrance to green L30
            [1.0, 0.25, 0.0, 0.0],  # LDK to red L57
            [0.0, 0.26, 1.0, 0.0],  # bedroom to blue L40
            [0.83, 0.87, 0.0, 0.0],  # balcony to yellow L85
            [0.0, 0.81, 0.76, 0.0],
        ]
    )  # bathroom to cyan L75

    # make colors subtractive
    channel_to_rgba[1:6, 0:3] -= 1

    # put it on white
    fps_rgba = np.clip(
        np.array([1.0, 1.0, 1.0, 1.0]) + (np.array(fps) @ channel_to_rgba), 0, 1
    )
    return fps_rgba


def create_dataset(filepath, batch_size=8):

    # This works with arrays as well
    dataset = tf.data.TFRecordDataset(filepath, compression_type="GZIP")

    # Maps the parser on every filepath in the array. You can set the number of parallel loaders here
    dataset = dataset.map(_parse_function, num_parallel_calls=4)

    # make the floorplan one-hot-ish
    dataset = dataset.map(_onehot_fp, num_parallel_calls=4)

    # rescale to [-1, 1]
    dataset = dataset.map(_rescale_fp, num_parallel_calls=4)

    # This dataset will go on forever
    dataset = dataset.repeat()

    # Set the number of datapoints you want to load and shuffle
    dataset = dataset.shuffle(BUFFER_SIZE)

    # Set the batchsize
    dataset = dataset.batch(batch_size)

    return dataset

In [7]:
# dataset = create_dataset(path_full_tfrecord)
# dataset_iter = tf.compat.v1.data.make_one_shot_iterator(dataset)
# plt.imshow(_visualize_fp_onehot(next(dataset_iter)[0][0]))

# DCGAN

In [8]:
class DCGAN_fp:
    def __init__(self, dir_save="dcgan"):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 6
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.dir_save = dir_save

        pathlib.Path(self.dir_save).mkdir(parents=True, exist_ok=True)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(
            loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"]
        )

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid, name="combined")
        self.combined.summary()
        self.combined.compile(loss="binary_crossentropy", optimizer=optimizer)

    def build_generator(self):
        model = Sequential(name="generator")

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        model = Sequential(name="discriminator")

        model.add(
            Conv2D(
                32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"
            )
        )
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation="sigmoid"))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(
        self, path_tfrecord, epochs=201, initial_epoch=0, batch_size=128, save_interval=1,
    ):
        if initial_epoch:
            self.load_model(initial_epoch - 1)

        n_examples = sum(
            1 for _ in tf.data.TFRecordDataset(path_tfrecord, compression_type="GZIP")
        )
        batches_per_epoch = math.ceil(n_examples / batch_size)

        # create a floorplan data iterator with size of half batch
        train_dataset = create_dataset(path_tfrecord, batch_size=batch_size // 2)
        dataset_iter = tf.compat.v1.data.make_one_shot_iterator(train_dataset)

        # Adversarial ground truths
        valid_full = np.ones((batch_size, 1))
        valid_double = np.ones((batch_size*2, 1))
        
        valid_half = np.ones((batch_size // 2, 1))
        fake_half = np.zeros((batch_size // 2, 1))

        # zero knowlegde guess for starting loss
        d_loss_fake = [-math.log(0.5), 0.5]
        g_loss = -math.log(0.5)

        # start time

        t0 = time.time()
        t1 = t0

        for epoch in range(initial_epoch, initial_epoch + epochs):
            for _ in range(batches_per_epoch):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Load a half batch of floorplan images
                imgs = next(dataset_iter)[0].numpy()

                # Sample noise and generate a half batch of new images
                noise = np.random.normal(0, 1, (batch_size // 2, self.latent_dim))
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator (real classified as ones and generated as zeros)
                d_loss_real = self.discriminator.train_on_batch(imgs, valid_half)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake_half)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Sample noise and generate a batch of new images
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Train the generator (wants discriminator to mistake images as real)
                g_loss = self.combined.train_on_batch(noise, valid_full)

            # If at save interval => save generated image samples
            if epoch % save_interval == 0 or epoch == initial_epoch + epochs - 1:

                # Plot the progress
                print(
                    "%d [D loss: %f, acc.: %.1f%%] [G loss: %f]"
                    % (epoch, d_loss[0], 100 * d_loss[1], g_loss),
                    end=" ",
                )

                self.save_imgs(epoch)

                t2 = time.time()
                print(f"{t2-t1:.1f}s elapsed since, total {t2-t0:.1f}s")
                t1 = t2

        # save model after finish
        self.save_model(epoch)

    def _visualize_fp(self, fps):
        # adjusted for different luminance
        channel_to_rgba = np.array(
            [
                [0.0, 0.0, 0.0, 1.0],  # unit mask to alpha
                [0.0, 0.33, 0.0, 0.0],  # entrance to green L30
                [1.0, 0.25, 0.0, 0.0],  # LDK to red L57
                [0.0, 0.26, 1.0, 0.0],  # bedroom to blue L40
                [0.83, 0.87, 0.0, 0.0],  # balcony to yellow L85
                [0.0, 0.81, 0.76, 0.0],
            ]
        )  # bathroom to cyan L75

        # make colors subtractive
        channel_to_rgba[1:6, 0:3] -= 1

        # put it on transparent white
        fps_rgba = np.clip(
            np.array([1.0, 1.0, 1.0, 0.0]) + (np.array(fps) @ channel_to_rgba), 0, 1
        )
        return fps_rgba

    def _visualize_fp_onehot(self, fps):
        # adjusted for different luminance
        channel_to_rgba = np.array(
            [
                [0.0, 0.0, 0.0, 0.0],  # ignore outdoor mask
                [0.0, 0.33, 0.0, 0.0],  # entrance to green L30
                [1.0, 0.25, 0.0, 0.0],  # LDK to red L57
                [0.0, 0.26, 1.0, 0.0],  # bedroom to blue L40
                [0.83, 0.87, 0.0, 0.0],  # balcony to yellow L85
                [0.0, 0.81, 0.76, 0.0],
            ]
        )  # bathroom to cyan L75

        # make colors subtractive
        channel_to_rgba[1:6, 0:3] -= 1

        # put it on white
        fps_rgba = np.clip(
            np.array([1.0, 1.0, 1.0, 1.0]) + (np.array(fps) @ channel_to_rgba), 0, 1
        )
        return fps_rgba

    def save_imgs(self, epoch, r=5, c=5):
        ### create a fixed noise
        # save the current state
        random_state = np.random.get_state()
        # set same noise for images
        np.random.seed(1106)
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        # reset the state
        np.random.set_state(random_state)

        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        # visualize as rgba
        gen_imgs_rgba = self._visualize_fp_onehot(gen_imgs)
        # gen_imgs_rgba = self._visualize_fp(gen_imgs)

        fig, axs = plt.subplots(r, c, figsize=(2, 2), dpi=300)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs_rgba[cnt, :, :, :])
                axs[i, j].axis("off")
                cnt += 1
        fig.savefig(
            f"{self.dir_save}/dcgan_{epoch:06}.png", bbox_inches="tight", pad_inches=0
        )
        plt.close()

    def save_model(self, end_epoch):
        for path, model in zip(
            [
                f"{self.dir_save}/dcgan_{end_epoch:06}_{name}.h5"
                for name in ["gen", "disc"]
            ],
            [self.generator, self.discriminator],
        ):
            with h5py.File(path, "w") as file:
                weight = model.get_weights()
                for i in range(len(weight)):
                    file.create_dataset("weight" + str(i), data=weight[i])

    def load_model(self, end_epoch):
        for path, model in zip(
            [
                f"{self.dir_save}/dcgan_{end_epoch:06}_{name}.h5"
                for name in ["gen", "disc"]
            ],
            [self.generator, self.discriminator],
        ):
            with h5py.File(path, "r") as file:
                weight = []
                for i in range(len(file.keys())):
                    weight.append(file["weight" + str(i)][:])

            model.set_weights(weight)

In [9]:
dcgan = DCGAN_fp(dir_save)

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 32)        1760      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 8, 8, 64)        

In [10]:
dcgan.train(
    path_full_tfrecord,
    epochs=1000,
    initial_epoch=300,
    batch_size=128,
    save_interval=10,
)

# 128*1000 -> less than 202s (now skip training on overpowering one)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
300 [D loss: 0.014234, acc.: 100.0%] [G loss: 5.019443] 25.1s elapsed since, total 25.1s
310 [D loss: 0.032046, acc.: 99.2%] [G loss: 6.159709] 227.6s elapsed since, total 252.6s
320 [D loss: 0.023849, acc.: 100.0%] [G loss: 5.359635] 228.3s elapsed since, total 480.9s
330 [D loss: 0.040728, acc.: 99.2%] [G loss: 5.766985] 228.2s elapsed since, total 709.0s
340 [D loss: 0.015531, acc.: 100.0%] [G loss: 4.915357] 228.4s elapsed since, total 937.4s
350 [D loss: 0.080299, acc.: 96.9%] [G loss: 4.749314] 228.6s elapsed since, total 1166.0s
360 [D loss: 0.057203, acc.: 98.4%] [G loss: 8.251804] 228.4s elapsed since, total 1394.4s
370 [D loss: 0.670255, acc.: 65.6%] [G loss: 4.112588] 228.5s elapsed since, total 1622.9s
380 [D loss: 0.418001, acc.: 82.0%] [G loss: 3.212224] 228.5s elapsed since, total 1851.4s
390 [D loss: 0.151341, acc.: 95.3%] [G loss: 9.040207] 228.3s elapsed since, total 2079.7s


1150 [D loss: 0.006007, acc.: 100.0%] [G loss: 7.913059] 229.3s elapsed since, total 19433.4s
1160 [D loss: 0.041196, acc.: 98.4%] [G loss: 7.040613] 228.4s elapsed since, total 19661.8s
1170 [D loss: 0.045575, acc.: 99.2%] [G loss: 8.322030] 228.4s elapsed since, total 19890.1s
1180 [D loss: 0.020733, acc.: 100.0%] [G loss: 5.542104] 227.3s elapsed since, total 20117.5s
1190 [D loss: 0.006270, acc.: 100.0%] [G loss: 10.060768] 227.7s elapsed since, total 20345.1s
1200 [D loss: 0.002221, acc.: 100.0%] [G loss: 6.351820] 227.8s elapsed since, total 20572.9s
1210 [D loss: 0.002263, acc.: 100.0%] [G loss: 10.253221] 227.9s elapsed since, total 20800.8s
1220 [D loss: 0.007871, acc.: 100.0%] [G loss: 10.900143] 228.4s elapsed since, total 21029.2s
1230 [D loss: 0.001089, acc.: 100.0%] [G loss: 10.958556] 228.3s elapsed since, total 21257.5s
1240 [D loss: 0.000332, acc.: 100.0%] [G loss: 10.216215] 227.8s elapsed since, total 21485.4s
1250 [D loss: 0.010889, acc.: 100.0%] [G loss: 6.453494] 